In [29]:
import requests
import pandas as pd
from datetime import datetime

In [2]:
def get_weather_data(api_key, lat, lon, start_time, end_time):
    base_url = "https://api.weatherbit.io/v2.0/history/subhourly"
    params = {
        "key": api_key,
        "lat": lat,
        "lon": lon,
        "start_date": start_time,
        "end_date": end_time,
        "tz": "local",  # Use local timezone
        "units": "metric",  # Use metric units for temperature and humidity
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    print(data)
    return data

In [23]:
def create_dataframe(data):
    df = pd.DataFrame(data['data'])
    
    # Extract time from timestamp_local and convert it to seconds
    df['time_seconds'] = (pd.to_datetime(df['timestamp_local']).dt.hour * 3600 +
                          pd.to_datetime(df['timestamp_local']).dt.minute * 60 +
                          pd.to_datetime(df['timestamp_local']).dt.second)

    df = df[['time_seconds', 'temp', 'rh', 'weather']]
    df['weather_description'] = df['weather'].apply(lambda x: x['description'] if 'description' in x else None)
    df.drop('weather', axis=1, inplace=True)
    df.rename(columns={'temp': 'temperature', 'rh': 'humidity'}, inplace=True)
    return df


In [21]:
api_key = "947141b2db6e415bb279750e5775f1ae"

# Replace with the coordinates of your location
latitude =12.8513
longitude =80.1470 

# Replace with your desired start and end times in the format 'YYYY-MM-DD:HH'
start_time = "2023-11-03:00"
end_time = "2023-11-10:00"

# Get weather data
weather_data = get_weather_data(api_key, latitude, longitude, start_time, end_time)

{'city_id': '1465868', 'city_name': 'Vengavasal', 'country_code': 'IN', 'data': [{'app_temp': 30.9, 'azimuth': 249, 'clouds': 50, 'dewpt': 25.6, 'dhi': 0, 'dni': 0, 'elev_angle': -84.1, 'ghi': 0, 'pod': 'n', 'precip_rate': 2, 'pres': 1010, 'revision_status': 'final', 'rh': 92, 'slp': 1011, 'snow_rate': 0, 'solar_rad': 0, 'temp': 27, 'timestamp_local': '2023-11-02T23:30:00', 'timestamp_utc': '2023-11-02T18:00:00', 'ts': 1698948000, 'uv': 0, 'vis': 4, 'weather': {'description': 'Light rain', 'code': 500, 'icon': 'r01n'}, 'wind_dir': 350, 'wind_gust_spd': 4.8, 'wind_spd': 2.4}, {'app_temp': 30.7, 'azimuth': 212.8, 'clouds': 43, 'dewpt': 25.2, 'dhi': 0, 'dni': 0, 'elev_angle': -83.3, 'ghi': 0, 'pod': 'n', 'precip_rate': 1.75, 'pres': 1009, 'revision_status': 'final', 'rh': 90, 'slp': 1011, 'snow_rate': 0, 'solar_rad': 0, 'temp': 27, 'timestamp_local': '2023-11-02T23:45:00', 'timestamp_utc': '2023-11-02T18:15:00', 'ts': 1698948900, 'uv': 0, 'vis': 4, 'weather': {'description': 'Light rain',

In [24]:
# Create DataFrame
weather_df = create_dataframe(weather_data)
weather_df.to_csv('weather_data_historic.csv')

In [25]:
weather_df

,time_seconds,temperature,humidity,weather_description
0,84600,27.0,92,Light rain
1,85500,27.0,90,Light rain
2,0,27.0,88,Light rain
3,900,27.0,88,Light rain
4,1800,27.0,88,Light rain
...,...,...,...,...
667,80100,26.0,88,Light rain
668,81000,26.0,94,Broken clouds
669,81900,26.0,94,Broken clouds
670,82800,26.0,94,Broken clouds


In [30]:
existing_df=pd.read_csv('weather_data.csv')
existing_df.columns=['temperature','humidity','light','weather_description','obs_time','time_seconds']
existing_df

,temperature,humidity,light,weather_description,obs_time,time_seconds
0,31.0,73.0,0.0,Haze,2023-11-11 09:19,2023-11-11 15:19:13
1,30.0,74.0,0.0,Haze,2023-11-11 09:30,2023-11-11 15:29:13
2,30.0,72.0,0.0,Haze,2023-11-11 09:39,2023-11-11 15:39:14
3,30.0,73.0,0.0,Haze,2023-11-11 10:00,2023-11-11 15:49:15
4,30.0,74.0,0.0,Haze,2023-11-11 10:00,2023-11-11 15:59:15
...,...,...,...,...,...,...
288,28.0,84.0,0.0,Fog,2023-11-14 01:41,2023-11-14 07:41:04
289,28.0,84.0,0.0,Fog,2023-11-14 02:00,2023-11-14 07:51:06
290,28.0,84.0,0.0,Fog,2023-11-14 02:01,2023-11-14 08:01:07
291,28.0,87.0,0.0,Overcast clouds,2023-11-14 11:13,2023-11-14 17:13:11


In [36]:
existing_df['time_seconds']=pd.to_datetime(existing_df["time_seconds"])

In [38]:
sensor_data= existing_df[['temperature','humidity','weather_description','time_seconds']]
sensor_data['time_seconds'] = (pd.to_datetime(sensor_data['time_seconds']).dt.hour * 3600 +
                          pd.to_datetime(sensor_data['time_seconds']).dt.minute * 60 +
                          pd.to_datetime(sensor_data['time_seconds']).dt.second)
sensor_data

C:\Users\sakth\AppData\Local\Temp\ipykernel_20136\168961909.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sensor_data['time_seconds'] = (pd.to_datetime(sensor_data['time_seconds']).dt.hour * 3600 +


,temperature,humidity,weather_description,time_seconds
0,31.0,73.0,Haze,55153
1,30.0,74.0,Haze,55753
2,30.0,72.0,Haze,56354
3,30.0,73.0,Haze,56955
4,30.0,74.0,Haze,57555
...,...,...,...,...
288,28.0,84.0,Fog,27664
289,28.0,84.0,Fog,28266
290,28.0,84.0,Fog,28867
291,28.0,87.0,Overcast clouds,61991


In [39]:
data=pd.concat([sensor_data, weather_df])
data

,temperature,humidity,weather_description,time_seconds
0,31.0,73.0,Haze,55153
1,30.0,74.0,Haze,55753
2,30.0,72.0,Haze,56354
3,30.0,73.0,Haze,56955
4,30.0,74.0,Haze,57555
...,...,...,...,...
667,26.0,88.0,Light rain,80100
668,26.0,94.0,Broken clouds,81000
669,26.0,94.0,Broken clouds,81900
670,26.0,94.0,Broken clouds,82800


In [18]:
data.to_csv('weather_data_full.csv',index=False)